# Mini-Project: SVM & LR Classification

There are two natural dependent variables in our dataset: label and attack_cat
Initial approach is to prep the existing variables for classification and throw in the "kitchen sink" and baseline our accuracy. Then look further to see what variables my be cross correlated, or less weight and remove them to see if we can increase our accuracy.

In this mini-project, we decided upon trying two different LR classifications and two different SVM classifications. The first we tries was a LR and SVM classification on the full dataset, variable "df" in our code and referred to as the "kitchen sink" model. We then tried doing the LR and SVM classification on the five most correlated features to the label feature, our dependent variable. We'll refer to this one as the "five-feature" model, which is represented by the variable "df_five"

In [24]:
# imports
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)

pd.set_option('display.max_columns', None)    # set the max columns to show to unlimited
%matplotlib inline

In [27]:
df = pd.read_csv('UNSW_NB15_training_set.csv', encoding='utf-8-sig')  # specifing encoding to get rid of the UTF- Byte order Mark (BOM) in the id field
df_five = pd.read_csv('miniLab.csv') # read in the five features most correlated to the label feature

df_five.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 6 columns):
sttl                82332 non-null int64
ct_dst_sport_ltm    82332 non-null int64
ct_src_dport_ltm    82332 non-null int64
swin                82332 non-null int64
dwin                82332 non-null int64
label               82332 non-null int64
dtypes: int64(6)
memory usage: 3.8 MB


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
id                   82332 non-null int64
dur                  82332 non-null float64
proto                82332 non-null object
service              82332 non-null object
state                82332 non-null object
spkts                82332 non-null int64
dpkts                82332 non-null int64
sbytes               82332 non-null int64
dbytes               82332 non-null int64
rate                 82332 non-null float64
sttl                 82332 non-null int64
dttl                 82332 non-null int64
sload                82332 non-null float64
dload                82332 non-null float64
sloss                82332 non-null int64
dloss                82332 non-null int64
sinpkt               82332 non-null float64
dinpkt               82332 non-null float64
sjit                 82332 non-null float64
djit                 82332 non-null float64
swin                 82332 non-n

In [14]:
# Lets remove attributes that are not useful to us during this first analysis pass
non_useful_features_list = ['id', 'attack_cat']
#non_useful_features_list = ['id'] # Re,pved attacl_cat/ LR accuracy: 0.754888861897
# id: n internal variable to just ref an obseration. deemed not usefl
# attack_cat: first try and just predict the label. 
#             It will obviously 1:1 correlate with label
#             We can circle back and swap it out with label 
#             to see if we get any better accuracy on an 
#             on an attack type level
for feature in non_useful_features_list:
    if feature in df:
        df.drop(feature, axis=1, inplace=True)  # Lets drop id as it is an internal variable to just ref an obseratio

# Overwrite the existing dataframe with the new dataframe that does not contain the 
# four unwanted records and confirm we have 4 less records (shold have 82328 observations)
if "is_ftp_login" in df:
    df = df[df.is_ftp_login != 2]
    if len(df) == 82328:
        print "duplicate record deleted successfully: " + str(len(df)) + " observations remaining" 

# Check to see if non useful features still exist in dataframe, if so, we did something wrong
for feature in non_useful_features_list:
    if feature in df:
        print "[" + feature + "]" + "still found, check removal code. (Should not see this)"

duplicate record deleted successfully: 82328 observations remaining


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82328 entries, 0 to 82331
Data columns (total 43 columns):
dur                  82328 non-null float64
proto                82328 non-null object
service              82328 non-null object
state                82328 non-null object
spkts                82328 non-null int64
dpkts                82328 non-null int64
sbytes               82328 non-null int64
dbytes               82328 non-null int64
rate                 82328 non-null float64
sttl                 82328 non-null int64
dttl                 82328 non-null int64
sload                82328 non-null float64
dload                82328 non-null float64
sloss                82328 non-null int64
dloss                82328 non-null int64
sinpkt               82328 non-null float64
dinpkt               82328 non-null float64
sjit                 82328 non-null float64
djit                 82328 non-null float64
swin                 82328 non-null int64
stcpb                82328 non-n

In [17]:
df.describe(include=['O']) # Surround in try except incase there are no object type features

,proto,service,state
count,82328,82328,82328
unique,131,13,7
top,tcp,-,FIN
freq,43091,47153,39335


Based on this, lets one-hot encode the above categorical variables. Note expect number of
features to increase based on sum of unique values. so $$43 + 131 + 13 + 7 - 3 = 191$$
Must subtract $3$ for the 3 original colums that exist

In [19]:
# http://stackoverflow.com/questions/19482970/get-list-from-pandas-dataframe-column-headers

# Surrounding code in try/except on case where there are no object type features to one-hot encode
try:
    tmp_df = df.describe(include=['O'])  # creates a temporary df with just categorical features that are of object type
    categorical_object_col_name_list = tmp_df.columns.values.tolist()
    for col_name in categorical_object_col_name_list:
        #print col_name
        tmp_df = pd.get_dummies(df[col_name], prefix=col_name)
        df = pd.concat((df,tmp_df), axis=1)
        df.drop(col_name, axis=1, inplace=True)  # go ahead and drop original feature as it has now been one-hot encoded
except ValueError as e:
    print "Value error({0}): ".format(e)  # Note

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82328 entries, 0 to 82331
Columns: 191 entries, dur to state_RST
dtypes: float64(162), int64(29)
memory usage: 120.6 MB


In [22]:
# we want to predict the X and y data as follows:
if 'label' in df:
    y = df['label'].values # get the labels we want
    del df['label'] # get rid of the class label
    X = df.values # use everything else to predict!

    # X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    # have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
# of the object and set it up. This object will be able to split our data into 
# training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n=num_instances,
                         n_iter=num_cv_iterations,
                         test_size  = 0.2)
                         
print cv_object

ShuffleSplit(82328, n_iter=3, test_size=0.2, random_state=None)


In [25]:
# first we create a reusable logisitic regression object
# here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object


accuracies = cross_val_score(lr_clf, X, y=y, cv=cv_object) # this also can help with parallelism
print(accuracies)
print "Average Accuracy across " + str(num_cv_iterations) + " shuffle split cross validation iterations = " + str(np.average(accuracies))

[ 0.75877566  0.7575003   0.75634641]
Average Accuracy across 3 shuffle split cross validation iterations = 0.757540791125


In [28]:
# we want to predict the X and y data as follows for df_five:
if 'label' in df_five:
    y = df_five['label'].values # get the labels we want
    del df_five['label'] # get rid of the class label
    X = df_five.values # use everything else to predict!

    # X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    # have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
# of the object and set it up. This object will be able to split our data into 
# training and testing splits
num_cv_iterations = 3
num_instances_five = len(y)
cv_object_five = ShuffleSplit(n=num_instances_five,
                         n_iter=num_cv_iterations,
                         test_size  = 0.2)
                         
# first we create a reusable logisitic regression object
# here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object


accuracies_five = cross_val_score(lr_clf, X, y=y, cv=cv_object_five) # this also can help with parallelism
print(accuracies_five)
print "Average Accuracy across " + str(num_cv_iterations) + " shuffle split cross validation iterations = " + str(np.average(accuracies))

[ 0.76783871  0.76632052  0.77014635]
Average Accuracy across 3 shuffle split cross validation iterations = 0.757540791125
